**Processo para enviar os e-mails ao cliente sobre seu consumo, quando tiver acima da média**

In [12]:
import pandas as pd
import oracledb
import warnings
warnings.filterwarnings("ignore")
import random
from datetime import datetime, timedelta

credencial_banco = {
    'dsn': 'oracle.fiap.com.br:1521/orcl',
    'usuario': 'rm553472',
    'senha': '100593'
}

def conectar(usuario, senha, dsn):
    try:
        conexao = oracledb.connect(user=usuario, password=senha, dsn=dsn, mode=oracledb.DEFAULT_AUTH)
        print("Conexão com o banco de dados Oracle estabelecida com sucesso.")
        return conexao
    except oracledb.DatabaseError as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

conexao = conectar(usuario='rm553472',senha='100593', dsn='oracle.fiap.com.br:1521/orcl')

query_alerta = """
select * from "RM553472"."T_Alerta_Consumo"
"""

df_alerta = pd.read_sql(query_alerta, conexao)
df_alerta

Conexão com o banco de dados Oracle estabelecida com sucesso.


,Id,IdUsuario,Ambiente,ConsumoMinimo,EmailDestino,Ativo,Descricao,TipoAlerta
0,1,1,geladeira,10,claudio_cssp@hotmail.com,sim,None,diario


**Coletar dados de consumo, agrupar e calcular a varição do consumo**

In [21]:

query_consumo = """
select * from "RM553472"."T_Consumo"
order by "DataConsumo" asc
"""

df_consumo = pd.read_sql(query_consumo, conexao)
df_consumo

,IdConsumo,IdUsuario,Comodo,Item,ConsumoDiario,DataConsumo,Valor
0,1,1,quarto1,arcondicionado,50,2024-11-21,10
1,3,1,Sala,Lâmpada,10,2024-11-21,100
2,4,1,Sala,Lâmpada,15,2024-11-22,120
3,5,1,Sala,Lâmpada,20,2024-11-23,140
4,6,1,Sala,Lâmpada,25,2024-11-24,160
5,7,1,Sala,Lâmpada,30,2024-11-25,180
6,8,1,Sala,Lâmpada,35,2024-11-26,200
7,9,1,Sala,Lâmpada,40,2024-11-27,220
8,10,1,Sala,Lâmpada,45,2024-11-28,240
9,11,1,Sala,Lâmpada,50,2024-11-29,260


In [25]:
df_consumo = df_consumo[['IdConsumo','IdUsuario','Comodo','Item','ConsumoDiario','DataConsumo']]

In [26]:
df = df_consumo
# Definindo a data de hoje
data_hoje = datetime.today().strftime('%Y-%m-%d')


In [27]:
# Calculando a variação de consumo
df['DataConsumo'] = pd.to_datetime(df['DataConsumo'])

df['ConsumoDiarioAnterior'] = df['ConsumoDiario'].shift(1)
df['VariaçãoConsumo'] = df['ConsumoDiario'] - df['ConsumoDiarioAnterior']


In [28]:
df

,IdConsumo,IdUsuario,Comodo,Item,ConsumoDiario,DataConsumo,ConsumoDiarioAnterior,VariaçãoConsumo
0,1,1,quarto1,arcondicionado,50,2024-11-21,NaN,NaN
1,3,1,Sala,Lâmpada,10,2024-11-21,50.0,-40.0
2,4,1,Sala,Lâmpada,15,2024-11-22,10.0,5.0
3,5,1,Sala,Lâmpada,20,2024-11-23,15.0,5.0
4,6,1,Sala,Lâmpada,25,2024-11-24,20.0,5.0
5,7,1,Sala,Lâmpada,30,2024-11-25,25.0,5.0
6,8,1,Sala,Lâmpada,35,2024-11-26,30.0,5.0
7,9,1,Sala,Lâmpada,40,2024-11-27,35.0,5.0
8,10,1,Sala,Lâmpada,45,2024-11-28,40.0,5.0
9,11,1,Sala,Lâmpada,50,2024-11-29,45.0,5.0


In [30]:
df['ConsumoDiarioAnterior'] = df['ConsumoDiarioAnterior'].fillna(0)
df['VariaçãoConsumo'] = df['VariaçãoConsumo'].fillna(0)

In [31]:
# Criando a análise com base na variação
df['Recomendação'] = df.apply(lambda row: f"Análise realizada em {data_hoje}. O consumo diário aumentou em {row['VariaçãoConsumo']} kws. Considerando que o consumo está aumentando, é recomendado substituir a lâmpada antiga por uma lâmpada LED, o que reduziria o consumo diário em aproximadamente {round(row['VariaçãoConsumo'] * 0.3)} reais e reduziria o consumo mensal em até {round(row['VariaçãoConsumo'] * 0.3 * 30)} reais.", axis=1)

In [35]:

# Selecionando as colunas finais
nova_tabela = df[['IdConsumo', 'Comodo', 'Item', 'ConsumoDiario', 'DataConsumo', 'ConsumoDiarioAnterior','VariaçãoConsumo', 'Recomendação']]


In [37]:
nova_tabela.columns

Index(['IdConsumo', 'Comodo', 'Item', 'ConsumoDiario', 'DataConsumo',
       'ConsumoDiarioAnterior', 'VariaçãoConsumo', 'Recomendação'],
      dtype='object')

In [38]:
nova_tabela

,IdConsumo,Comodo,Item,ConsumoDiario,DataConsumo,ConsumoDiarioAnterior,VariaçãoConsumo,Recomendação
0,1,quarto1,arcondicionado,50,2024-11-21,0.0,0.0,Análise realizada em 2024-11-21. O consumo diá...
1,3,Sala,Lâmpada,10,2024-11-21,50.0,-40.0,Análise realizada em 2024-11-21. O consumo diá...
2,4,Sala,Lâmpada,15,2024-11-22,10.0,5.0,Análise realizada em 2024-11-21. O consumo diá...
3,5,Sala,Lâmpada,20,2024-11-23,15.0,5.0,Análise realizada em 2024-11-21. O consumo diá...
4,6,Sala,Lâmpada,25,2024-11-24,20.0,5.0,Análise realizada em 2024-11-21. O consumo diá...
5,7,Sala,Lâmpada,30,2024-11-25,25.0,5.0,Análise realizada em 2024-11-21. O consumo diá...
6,8,Sala,Lâmpada,35,2024-11-26,30.0,5.0,Análise realizada em 2024-11-21. O consumo diá...
7,9,Sala,Lâmpada,40,2024-11-27,35.0,5.0,Análise realizada em 2024-11-21. O consumo diá...
8,10,Sala,Lâmpada,45,2024-11-28,40.0,5.0,Análise realizada em 2024-11-21. O consumo diá...
9,11,Sala,Lâmpada,50,2024-11-29,45.0,5.0,Análise realizada em 2024-11-21. O consumo diá...


**Inserir os dados no banco e utilizar na tabela do .net para mostrar as notificações que foram enviadas**

In [43]:
linhas = nova_tabela.itertuples(index=False, name=None)

In [50]:
conexao = conectar(usuario='rm553472',senha='100593', dsn='oracle.fiap.com.br:1521/orcl')


# Inserir os dados no banco
try:
    with conexao.cursor() as cursor:
        for linha in linhas:
            query = """
            INSERT INTO "RM553472"."T_Alerta_Gerado" (
                "IdConsumo", "Comodo", "Item", "ConsumoDiario", "DataConsumo",
                "ConsumoDiarioAnterior", "VariacaoConsumo", "Recomendacao"
            ) VALUES (:1, :2, :3, :4, TO_DATE(:5, 'YYYY-MM-DD'), :6, :7, :8)
            """
            cursor.execute(query, linha)

    # Confirmar as mudanças
    conexao.commit()
    print("Dados inseridos com sucesso!")

except oracledb.DatabaseError as e:
    print("Erro ao inserir os dados:", e)

finally:
    conexao.close()


Conexão com o banco de dados Oracle estabelecida com sucesso.
Dados inseridos com sucesso!


In [51]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def enviar_email():
    # Dados do Gmail
    email_user = 'cssp.claudio@gmail.com'  
    email_password = 'afdz ihed ysxa nbvx' 
    email_send = 'claudio_cssp@hotmail.com' 

    # Configurar o servidor SMTP do Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)  
    server.starttls() 
    server.login(email_user, email_password) 

    # Criar a mensagem do e-mail
    subject = 'Alerta - Aumento do consumo de energia'
    body = 'Indentificações um aumento no seu consumo de energia que pode ser prejudicial e acarretar em custos extras. Também percebemos que está na hora de trocar sua lampada!'

    msg = MIMEMultipart()
    msg['From'] = email_user
    msg['To'] = email_send
    msg['Subject'] = subject

    # Anexar o corpo do e-mail
    msg.attach(MIMEText(body, 'plain'))

    # Enviar o e-mail
    server.sendmail(email_user, email_send, msg.as_string())

    # Fechar a conexão com o servidor
    server.quit()

    print("E-mail enviado com sucesso!")

# Chamar a função para enviar o e-mail
enviar_email()


E-mail enviado com sucesso!
